In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
import sender

In [2]:
sender_encoder = sender.Sender(num_options=10, batch_size=32)
sender_only_target = sender.SenderOnlyTarget(batch_size=32)
sender = sender.Sender_LSTM(vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=True)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [3]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [4]:
input_concepts, sender_input, targets, receiver_input = data.getInput()
#print(sender_input.shape)

shape sender input:
(32, 10, 1, 595)


In [5]:
sender_encoded = sender_encoder(sender_input)
message, logits, entropy, state_h = sender(sender_encoded)
print("message:")
print(message)
print("logits:")
print(logits)
print("entropy:")
print(entropy)
print("state_h:")
print(state_h)

32
input-shape:
(32, 10, 44)
message:
tf.Tensor(
[[59 11 10  9 36 62 54 36 60  0  0  0  0  0  0  0  0  0]
 [93 64 83 47 70 17 18 93 21  0  0  0  0  0  0  0  0  0]
 [84 59  5 73 70 94 41 38 67  0  0  0  0  0  0  0  0  0]
 [60 81 75 97 76 41 89 41 65  0  0  0  0  0  0  0  0  0]
 [81 24 79 17 16 55 77 95  4  0  0  0  0  0  0  0  0  0]
 [90 16 80 10 10 38 75  3  8  0  0  0  0  0  0  0  0  0]
 [61 76  0 13 71 42  2 87 57  0  0  0  0  0  0  0  0  0]
 [84 87 47 43 33 84 60 51 15  0  0  0  0  0  0  0  0  0]
 [92 53 83 90 48 71 86 73  3  0  0  0  0  0  0  0  0  0]
 [73 36 77 46 15 56 29 70 56  0  0  0  0  0  0  0  0  0]
 [79 44 38 91 63 45 79 52 87  0  0  0  0  0  0  0  0  0]
 [22 92 63 67 50 29  7 10  7  0  0  0  0  0  0  0  0  0]
 [25 28 57 74 51 92 87 83 70  0  0  0  0  0  0  0  0  0]
 [10 80 76  5 98 92  8 28 46  0  0  0  0  0  0  0  0  0]
 [94 52 49  5 78 16 56  2 15  0  0  0  0  0  0  0  0  0]
 [96 54 53 37 95  0 44 73 24  0  0  0  0  0  0  0  0  0]
 [ 6 89 86 10 39 48 13 61 47  0  0  0  

In [6]:
sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input)

receiver_output = []
for i in range(32):
    receiver_output.append(input_concepts[i][sample[i]])
    
print(receiver_output)

10


InvalidArgumentError: In[0] and In[1] must have compatible batch dimensions: [10,10,1,44] vs. [32,44,1] [Op:BatchMatMulV2]